In [1]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('../dataset/dataset.csv')

# ---------------------------
# 1. Data Loading & Parsing
# ---------------------------
def parse_signal(signal_str):
    """Convert complex string to magnitude/phase features"""
    cleaned = signal_str.strip('[]').replace(' ', '')
    parts = cleaned.split('),(')
    signal = []
    for p in parts:
        p = p.replace('(', '').replace(')', '')
        try:
            cmplx = complex(p)
            signal.append([abs(cmplx), np.angle(cmplx)])  # Magnitude + Phase
        except ValueError:
            continue
    return np.array(signal[:208])  # Shape: (208, 2)

def parse_secret_code(code_str):
    """Convert secret code string to integer array"""
    return np.array([int(x) for x in code_str.strip('[]').split(', ')])


# Parse all columns
X_signal = np.array([parse_signal(s) for s in df['received_signal']])  # (15000, 208, 2)
X_secret = np.array([parse_secret_code(c) for c in df['secret_code']])  # (15000, 13)
y = df[['jet1_x', 'jet1_y', 'jet1_z', 'jet2_x', 'jet2_y', 'jet2_z']].values  # (15000, 6)

In [2]:
print(X_signal[:1])

[[[ 0.2229913  -1.00620656]
  [ 1.02854609 -0.58463834]
  [ 1.33988282 -3.05400802]
  [ 0.95439686 -2.12703261]
  [ 2.39794737 -2.85318375]
  [ 2.41176324 -2.94039406]
  [ 1.88649359 -2.73590844]
  [ 2.09875628  3.05424141]
  [ 1.75956874  2.84119159]
  [ 2.08312458  3.0230854 ]
  [ 2.33714127  2.83773895]
  [ 1.91157554  2.64836719]
  [ 1.00369649  2.98933202]
  [ 1.71609921  2.66619113]
  [ 2.39077512  2.67200443]
  [ 1.92979245  2.49660498]
  [ 2.18216289  2.3755062 ]
  [ 1.60648387  2.39479077]
  [ 1.56592539  1.85281706]
  [ 2.41254537  1.72659667]
  [ 1.8774701   1.67987811]
  [ 2.01993824  1.74278086]
  [ 2.37840474  1.6988247 ]
  [ 1.9292007   1.15051429]
  [ 1.94939897  1.63812123]
  [ 2.18241022  1.13626155]
  [ 2.01916819  1.27066915]
  [ 1.58849702  1.27336048]
  [ 1.73513605  1.39274444]
  [ 1.85123951  1.25285589]
  [ 2.0786206   0.8912264 ]
  [ 2.23254926  0.93281102]
  [ 2.61655436  0.78950846]
  [ 3.44359385  0.7027852 ]
  [ 3.95310036  0.27192271]
  [ 3.16634329  0.50

In [3]:
X_signal

array([[[ 0.2229913 , -1.00620656],
        [ 1.02854609, -0.58463834],
        [ 1.33988282, -3.05400802],
        ...,
        [ 2.75443118,  2.62664116],
        [ 3.07722138,  3.11006478],
        [ 2.73787314,  2.88797034]],

       [[ 0.03664518,  2.84340005],
        [ 0.93725984,  3.03741696],
        [ 1.07769295, -3.03040808],
        ...,
        [ 0.58892108, -0.55870694],
        [ 1.20786625, -0.79975285],
        [ 2.28187429, -0.82649512]],

       [[ 0.05200478, -1.92108213],
        [ 1.19081254,  1.82941536],
        [ 1.84373096,  1.92010019],
        ...,
        [ 2.61126618,  1.88178862],
        [ 2.71345309,  1.89208373],
        [ 2.57725522,  1.96217491]],

       ...,

       [[ 0.33244008, -0.97404419],
        [ 0.56092625, -1.18170298],
        [ 0.17141353, -2.26017415],
        ...,
        [ 3.36215329, -0.58652275],
        [ 3.62771652, -0.83170005],
        [ 3.90762244, -1.13072922]],

       [[ 0.38598102, -0.74085196],
        [ 1.44245481, -0.69

In [4]:
X_signal.shape

(15000, 208, 2)

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------------------------
# 2. Data Splitting & Normalization
# ---------------------------
# First split: 80% train, 20% temp
X_sig_train_raw, X_sig_temp_raw, X_sec_train_raw, X_sec_temp_raw, y_train_raw, y_temp_raw = train_test_split(
    X_signal, X_secret, y, test_size=0.2, random_state=42
)

# Second split: 50% validation, 50% test
X_sig_val_raw, X_sig_test_raw, X_sec_val_raw, X_sec_test_raw, y_val_raw, y_test_raw = train_test_split(
    X_sig_temp_raw, X_sec_temp_raw, y_temp_raw, test_size=0.5, random_state=42
)

# Signal Normalizer (fit on training only)
scaler_signal = StandardScaler()
X_sig_train_flat = X_sig_train_raw.reshape(-1, 2)
scaler_signal.fit(X_sig_train_flat)

def scale_and_reshape(X_raw, scaler):
    X_flat = X_raw.reshape(-1, 2)
    X_scaled = scaler.transform(X_flat)
    return X_scaled.reshape(-1, 208, 2)

X_sig_train = scale_and_reshape(X_sig_train_raw, scaler_signal)
X_sig_val = scale_and_reshape(X_sig_val_raw, scaler_signal)
X_sig_test = scale_and_reshape(X_sig_test_raw, scaler_signal)

# Secret Code Normalizer
scaler_secret = StandardScaler()
scaler_secret.fit(X_sec_train_raw)

X_sec_train = scaler_secret.transform(X_sec_train_raw)
X_sec_val = scaler_secret.transform(X_sec_val_raw)
X_sec_test = scaler_secret.transform(X_sec_test_raw)

# Target Normalizer
scaler_target = StandardScaler()
scaler_target.fit(y_train_raw)

y_train = scaler_target.transform(y_train_raw)
y_val = scaler_target.transform(y_val_raw)
y_test = scaler_target.transform(y_test_raw)


# --------------------------------------------------
# 4. Verification
# --------------------------------------------------
print("Training shapes:")
print(f"Signals: {X_sig_train.shape}, Codes: {X_sec_train.shape}, Targets: {y_train.shape}")
print("\nValidation shapes:")
print(f"Signals: {X_sig_val.shape}, Codes: {X_sec_val.shape}, Targets: {y_val.shape}")
print("\nTest shapes:")
print(f"Signals: {X_sig_test.shape}, Codes: {X_sec_test.shape}, Targets: {y_test.shape}")

Training shapes:
Signals: (12000, 208, 2), Codes: (12000, 13), Targets: (12000, 6)

Validation shapes:
Signals: (1500, 208, 2), Codes: (1500, 13), Targets: (1500, 6)

Test shapes:
Signals: (1500, 208, 2), Codes: (1500, 13), Targets: (1500, 6)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import BatchNormalization, GlobalMaxPooling1D, Dropout, Embedding, Dot, Softmax
from tensorflow.keras import layers
import tensorflow as tf
import math

# ---------------------------
# 3. Model Architecture
# ---------------------------
def cosine_annealing(epoch, lr):
    """Learning rate scheduler"""
    return lr * 0.5 * (1 + math.cos(math.pi * epoch / 50))

def weighted_mse(y_true, y_pred):
    """Custom loss with higher weight for altitude (z) coordinates"""
    weights = tf.constant([1.0, 1.0, 2.0, 1.0, 1.0, 2.0])  # Double weight for z-coordinates
    return tf.reduce_mean(tf.square(y_true - y_pred) * weights)

# Signal Branch (CNN)
signal_input = Input(shape=(208, 2), name='signal_input')
x = Conv1D(64, 3, activation='relu', padding='same')(signal_input)
x = BatchNormalization()(x)
x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)  # Now 104 time steps
x = Conv1D(256, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = GlobalMaxPooling1D()(x)  # Reduces to (256,)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)

# Secret Code Branch (Dense)
code_input = Input(shape=(13,), name='code_input')  # Fixed shape definition
y = Dense(64, activation='relu')(code_input)
y = BatchNormalization()(y)
y = Dense(64, activation='relu')(y)
y = BatchNormalization()(y)
y = Dense(64, activation='relu')(y)  # Final shape (batch, 64)

# Attention-Based Fusion (Fixed implementation)
# Project both branches to same dimension
x_proj = layers.Dense(64)(x)  # From 256 → 64
y_proj = layers.Dense(64)(y)  # Ensure same dimension

# Compute attention scores using additive attention
attention_scores = layers.Add()([
    layers.Dense(64)(x_proj), 
    layers.Dense(64)(y_proj)
])
attention_weights = layers.Activation('softmax')(attention_scores)

# Apply attention weights to x_proj features
context_vector = layers.Multiply()([x_proj, attention_weights])  # (None, 64)

# Final merge with code features
merged = layers.concatenate([context_vector, y_proj])  # (None, 128)

# Final Layers
z = Dense(512, activation='relu')(merged)
z = BatchNormalization()(z)
z = Dense(256, activation='relu')(z)
z = BatchNormalization()(z)
z = Dense(128, activation='relu')(z)
outputs = Dense(6, activation='linear')(z)

# Model Compilation
model = Model(inputs=[signal_input, code_input], outputs=outputs)
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss=weighted_mse,
    metrics=['mae']
)

# --------------------------------------------------
# Reconnaissance Report (Model Summary)
# --------------------------------------------------
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 signal_input (InputLayer)   [(None, 208, 2)]             0         []                            
                                                                                                  
 conv1d_15 (Conv1D)          (None, 208, 64)              448       ['signal_input[0][0]']        
                                                                                                  
 batch_normalization_29 (Ba  (None, 208, 64)              256       ['conv1d_15[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 conv1d_16 (Conv1D)          (None, 208, 128)             24704     ['batch_normalization_29

In [18]:
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=20,
    mode='min',
    restore_best_weights=True
)

lr_scheduler = LearningRateScheduler(cosine_annealing)

In [19]:
import tensorflow as tf
def add_noise(signal, noise_level=0.01):
    return signal + np.random.normal(0, noise_level, signal.shape)

# Create noisy training data
X_sig_train_noisy = add_noise(X_sig_train)

# Train the model with the updated configuration
history = model.fit(
    x=[X_sig_train_noisy, X_sec_train],
    y=y_train,
    validation_data=([X_sig_val, X_sec_val], y_val),
    epochs=50,
    batch_size=128,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)


Epoch 1/50
94/94 [==============================] - 17s 163ms/step - loss: 1.4526 - mae: 0.8811 - val_loss: 1.3452 - val_mae: 0.8683 - lr: 0.0010
Epoch 2/50
94/94 [==============================] - 14s 150ms/step - loss: 1.2815 - mae: 0.8175 - val_loss: 1.2772 - val_mae: 0.8390 - lr: 9.9901e-04
Epoch 3/50
94/94 [==============================] - 15s 155ms/step - loss: 1.2518 - mae: 0.8070 - val_loss: 1.2562 - val_mae: 0.8223 - lr: 9.9507e-04
Epoch 4/50
94/94 [==============================] - 14s 152ms/step - loss: 1.2564 - mae: 0.8060 - val_loss: 1.2347 - val_mae: 0.8076 - lr: 9.8626e-04
Epoch 5/50
94/94 [==============================] - 15s 158ms/step - loss: 1.2365 - mae: 0.7999 - val_loss: 1.1953 - val_mae: 0.7944 - lr: 9.7077e-04
Epoch 6/50
94/94 [==============================] - 17s 184ms/step - loss: 1.2295 - mae: 0.7943 - val_loss: 1.1984 - val_mae: 0.7910 - lr: 9.4701e-04
Epoch 7/50
94/94 [==============================] - 16s 168ms/step - loss: 1.2117 - mae: 0.7895 - val_lo

In [21]:
test_loss, test_mae = model.evaluate(  
    [X_sig_test, X_sec_test],   
    y_test,  
    verbose=0  
)  
print(f"Test MSE: {test_loss:.4f} (Normalized)")  
print(f"Test MAE: {test_mae:.4f} (Normalized)")  

# Inverse-transform for real-world error  
y_pred_normalized = model.predict([X_sig_test, X_sec_test])  
y_pred_real = scaler_target.inverse_transform(y_pred_normalized)  
y_test_real = scaler_target.inverse_transform(y_test)  

# Calculate real-world MAE  
mae_real = np.mean(np.abs(y_pred_real - y_test_real))  
print(f"Real-World MAE: {mae_real:.2f} meters")  

Test MSE: 1.1640 (Normalized)
Test MAE: 0.7692 (Normalized)
47/47 [==============================] - 1s 13ms/step
Real-World MAE: 5536.95 meters
